In [32]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('..', 'TFIDF'))
sys.path.append(os.path.join('..', 'util'))
from timer import Timer
from sklearn import metrics
import webbrowser
import time

In [2]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [41]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:10000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_sm.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_sm.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [4]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
# tfidf.calculate_tfidf_norms()
serial_cossims = tfidf.calculate_cossims()
serial_simhashes = tfidf.calculate_simhashes()
serial_distances = tfidf.calculate_distances()

Time for initialization: 1.00135803223e-05
Time for load_questions: 9.53674316406e-07
Time for load_indices: 0.00335597991943
Time for init_tfs: 0.0243649482727
Time for create_tfs: 0.131696939468
Time for create_idf: 0.184086799622
Time for calculate_tfidfs: 0.0182771682739
Time for calculate_cossims: 0.205546855927
Time for calculate_simhashes: 8.99054408073
Time for calculate_distances: 0.303831100464


In [5]:
serial_cossims.shape

(1000, 1000)

In [42]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 64)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances = np.asarray(tfidf_c.calculate_distances())

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 2.98023223877e-05
Time for load_questions: 0.111461877823
Time for load_indices: 0.0291638374329
Time for init_tfs: 0.688503980637
Time for create_tfs: 0.144814014435
Time for calculate_idf: 0.205336093903
Time for int_tfidfs: 0.61073589325
Time for calculate_tfidfs: 0.113712072372
Time for calculate_simhashes: 0.369334936142
Time for calculate distances: 0.837061882019


In [7]:
# Verification
print np.linalg.norm(cython_tfs - serial_tfs)
print np.linalg.norm(cython_idf - serial_idf)
print np.linalg.norm(cython_tfidfs - serial_tfidfs)
print np.linalg.norm(cython_simhashes - serial_simhashes)
print np.linalg.norm(cython_distances - serial_distances)

0.0
5.66585172214e-06
8.73883976884e-05
0.0
0.0


In [ ]:
#The following are from https://yesteapea.wordpress.com/2013/03/03/counting-the-number-of-set-bits-in-an-integer/
def numBits64(i):
    i = i - ((i >> np.uint64(1)) & np.uint64(0x5555555555555555))
    i = (i & np.uint64(0x3333333333333333)) + ((i >> np.uint64(2)) & np.uint64(0x3333333333333333))
    i = ((i + (i >> np.uint64(4))) & np.uint64(0x0F0F0F0F0F0F0F0F))
    return (i*(np.uint64(0x0101010101010101)))>>np.uint64(56)

def numBits32(i):
    i = i - ((i >> 1) & 0x55555555)
    i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
    i = ((i + (i >> 4)) & 0x0F0F0F0F)
    return (i*(0x01010101))>>24

In [ ]:
A = np.array([serial_simhashes] * len(serial_simhashes))
B = np.copy(A)
distances = numBits64(A ^ B.T)

## Clustering

In [43]:
from sklearn.cluster import DBSCAN

In [44]:
db64 = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances)

The below will just see how many of our documents were NOT labeled as "noise"

In [45]:
sum(map(lambda x: 1 if x >= 0 else 0, db64.labels_))

338

How many labels do we have?

In [39]:
set(db64.labels_)

{-1, 0}

Now doing it for the 32BIT cython distances

In [ ]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(1, False, "coarse", 32)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = np.asarray(tfidf_c.create_tfs())[:,0:len(word_indices)]
cython_idf = np.asarray(tfidf_c.calculate_idf(len(word_indices)))[0:len(word_indices)]
tfidf_c.init_tfidfs()
cython_tfidfs = np.asarray(tfidf_c.calculate_tfidfs())[:,0:len(word_indices)]
cython_simhashes = np.asarray(tfidf_c.calculate_simhashes())
cython_distances32 = tfidf_c.calculate_distances()

In [ ]:
db32 = DBSCAN(metric="precomputed", eps=2, min_samples=3).fit(cython_distances32)

In [ ]:
sum(map(lambda x: 1 if x >= 0 else 0, db32.labels_))

Lastly, lets conduct DBSCAN on our Cosine Similarity matrix 

In [ ]:
dbCossim = DBSCAN(metric="precomputed", eps=0.9, min_samples=3).fit(serial_cossims)

In [ ]:
sum(map(lambda x: 1 if x >= 0 else 0, dbCossim.labels_))

In [ ]:
set(dbCossim.labels_)

In [ ]:
max(serial_cossims[0][1:])


In [ ]:
metrics.adjusted_rand_score(db.labels_, db32.labels_)

In [ ]:
metrics.adjusted_rand_score(dbCossim.labels_, db.labels_)

In [ ]:
metrics.homogeneity_score(dbCossim.labels_, db.labels_)

Below is the code that takes in a db instances and displays a particular cluster for it

In [40]:
def displayCluster(db, how_near=2, how_many=3, label=0, max_pages=5):
    all_one_labels = []
    counter = 0
    for i in xrange(len(db.labels_)):
        if db.labels_[i] == -1:
            counter += 1
        elif db.labels_[i] == label:
            all_one_labels.append(i)
    counter = 0 
    for q in all_one_labels:
        lnk = questions[q]['link']
        if counter >= max_pages:
            break
        counter += 1
        webbrowser.open_new(lnk)
        time.sleep(3)

In [46]:
displayCluster(db64, label=5)

In [50]:
all_one_labels = []
for i in xrange(len(db64.labels_)):
    if db64.labels_[i] == -1:
        counter += 1
    elif db64.labels_[i] == 5:
        all_one_labels.append(i)
print all_one_labels

[191, 6760, 6906]


In [51]:
for i in all_one_labels: 
    print " ".join(question_texts[i])
    print ""

i chang the color of the powershel and now i ca n't chang the color of the input text , is alway yellow . 1** i chang the color of the background and the color of the text chang color 2** the color of the background chang right but in the display text the color look yellow alway chang i can do someth to reset the color ?

there are 16 color in c # : is there ani way i can add more color ( like brown , orang , etc . ) ?

i want the color scheme to span complet across the termin boundari . i am use color scheme scroller plugin to switch between differ theme . i have upload a .gif file so that you can clearli see what i want to get fix . vim colorschem doe n't complet chang the color of editor . there are some termin color 's border left around the vim 's overridden color scheme . how would i fix it . pleas check the imag on thi link . stackoverflow doe n't allow upload an imag > 2mb

